In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
data = pd.read_csv('../DataSet/offensive_dataset.csv', encoding='utf-16')
data.head()

,Comment,is_off
0,شوف حد ين*** أنت و إيران.,1
1,شوف تيفي قوداتها مع الجن غي بلاتي يجي من مكة,0
2,عجب يتفاوظون على شعوب عربية وعليهم وهىلا تعلم,0
3,ههههههه هاد شوف تيفي اللي حطت عليه مشا فيها,0
4,الرزق يأتي من حيث لا تدري 🙌🏻❤️,0


In [9]:
# count of target classes
data['is_off'].value_counts()


1    7000
0    7000
Name: is_off, dtype: int64

In [10]:
# get the data with target = 1 (offensive)
offensive_data = data[data['is_off'] == 1]
offensive_data.head()


,Comment,is_off
0,شوف حد ين*** أنت و إيران.,1
5,دخلو سوق كركم \nشوفو شكون مكتب عليكم تعيشو عبي...,1
6,@User.IDX لسسه امبارح بحذركو من ولاد الجزمه في...,1
8,الله يعفو عليها مسكينة دغيا طارو بها سصحاب الح...,1
9,تفو شحااال حامضة هاد سعيييدة...معرفت علاه كيجي...,1


In [11]:
import string
import re
import nltk

In [12]:
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations = arabic_punctuations + english_punctuations


def remove_urls (text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return text


def remove_emails(text):
    text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", "",  text, flags=re.MULTILINE)
    return text

# def remove_emoji(text):
#     return emoji.get_emoji_regexp().sub(u'', text)

def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def normalization(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def remove_stopwords(text):
    filtered_sentence = [w for w in text.split() if not w in arabic_stopwords]
    return ' '.join(filtered_sentence)

def cleaning_content(line):
    if (isinstance(line, float)):
        return None
    line.replace('\n', ' ')
    line = remove_emails(line)
    line = remove_urls(line)
    line = remove_emoji(line)
    nline = [w if '@' not in w else 'USERID' for w in line.split()]
    line = ' '.join(nline)
    line = line.replace('RT', '').replace('<LF>', '').replace('<br />','').replace('&quot;', '').replace('<url>', '').replace('USERID', '')


    # add spaces between punc,
    line = line.translate(str.maketrans({key: " {0} ".format(key) for key in punctuations}))

    # then remove punc,
    translator = str.maketrans('', '', punctuations)
    line = line.translate(translator)

    line = remove_stopwords(line)
    line=remove_diacritics(normalization(line))
    return line

def hasDigits(s):
    return any( 48 <= ord(char) <= 57  or 1632 <= ord(char) <= 1641 for char in s)


In [13]:
offensive_data.Comment = offensive_data.Comment.apply(cleaning_content)

C:\Users\makch\AppData\Local\Temp\ipykernel_12600\845588348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_data.Comment = offensive_data.Comment.apply(cleaning_content)


In [26]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=arabic_stopwords)
dtm = cv.fit_transform(offensive_data['Comment'])

feature_names = cv.get_feature_names()
print(len(feature_names))

c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


11669


c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
LDA_model = LatentDirichletAllocation(n_components= 5, random_state=42 , max_iter=100, n_jobs=-1)

LDA_model.fit(dtm)

LatentDirichletAllocation(max_iter=100, n_components=5, n_jobs=-1,
                          random_state=42)

In [36]:
for i, topic in enumerate(LDA_model.components_):
    print("THE TOP {} WORDS FOR TOPIC #{}".format(20, i))
    print([cv.get_feature_names()[index] for index in topic.argsort()[-20:]])
    print("\n")

THE TOP 20 WORDS FOR TOPIC #0


c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['قبح', 'الحق', 'البرتوش', 'تعالي', 'ديال', 'او', 'قال', 'يوم', 'يقول', 'حول', 'قناه', 'قوه', 'والله', 'بالله', 'الناس', 'الجزيره', 'الي', 'ان', 'الا', 'الله']


THE TOP 20 WORDS FOR TOPIC #1
['علي', 'والله', 'داعش', 'اليمن', 'سوريا', 'او', 'انه', 'قتل', 'يعني', 'انت', 'الي', 'قناه', 'السعوديه', 'مصر', 'قطر', 'الشعب', 'الجزيره', 'ايران', 'الله', 'ان']


THE TOP 20 WORDS FOR TOPIC #2
['ان', 'اش', 'هاذ', 'الخدمه', 'دوله', 'محمد', 'السعوديه', 'اختي', 'جهنم', 'الدوله', 'اللهم', 'الاسلاميه', 'الا', 'العرب', 'لله', 'اكبر', 'الاسلام', 'الي', 'او', 'الله']


THE TOP 20 WORDS FOR TOPIC #3
['ليك', 'اللي', 'شوف', 'باش', 'اش', 'والله', 'انا', 'او', 'ماشي', 'ديال', 'سير', 'واش', 'ديالك', 'يعطيك', 'راه', 'شي', 'تفو', 'بنت', 'هاد', 'الله']


THE TOP 20 WORDS FOR TOPIC #4
['منهم', 'فابور', 'امريكا', 'انا', 'لماذا', 'اهل', 'انك', 'العربيه', 'اللي', 'اللهم', 'السيسي', 'المغرب', 'الا', 'مصر', 'العرب', 'او', 'اي', 'انت', 'الي', 'ان']




In [30]:
final_topic = LDA_model.transform(dtm)

offensive_data['Topic'] = final_topic.argmax(axis=1)

offensive_data.head()

C:\Users\makch\AppData\Local\Temp\ipykernel_12600\3345567195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_data['Topic'] = final_topic.argmax(axis=1)


,Comment,is_off,Topic
0,شوف حد ايران,1,3
5,دخلو سوق كركم شوفو شكون مكتب عليكم تعيشو عبيد ...,1,2
6,لسسه امبارح بحذركو ولاد الجزمه فيفي ولاد شرموط...,1,1
8,الله يعفو عليها مسكينه دغيا طارو سصحاب الحسنات...,1,3
9,تفو شحااال حامضه هاد سعيييده معرفت علاه كيجيبو...,1,3


In [33]:
# select the comments with topic 0
topic_0 = offensive_data[offensive_data['Topic'] == 0]
topic_0.head()


,Comment,is_off,Topic
24,,1,0
29,التقيه دين الرافضه لذلك العب غيرها ههههه,1,0
51,فساد صارخ زاءد ظلم فادح يفرخ ارهاب عفوي يمكن ت...,1,0
65,الله كدب حلوف حلوفه الغيس,1,0
106,بشعب يتعاطف زانيه قول كلنا ليلي يقبح زوجه محام...,1,0


In [35]:

topic_0.shape


(716, 3)

In [31]:
import pyLDAvis.sklearn

# Enable the visualization on the notebook
pyLDAvis.enable_notebook()

# Create the panel for the visualization
panel = pyLDAvis.sklearn.prepare(LDA_model, dtm, cv, mds='tsne') 

# Show the panel
panel

c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\makch\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\makch\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  wa

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153  -72.269554       1        1  41.491833
1       51.163364 -159.968018       2        1  17.234561
4      -73.144554   21.523815       3        1  15.702616
2      -66.410133 -110.606438       4        1  13.302414
0       54.417324   12.443444       5        1  12.268576, topic_info=         Term         Freq        Total Category  logprob  loglift
2365     الله  2429.000000  2429.000000  Default  30.0000  30.0000
9675      هاد   592.000000   592.000000  Default  29.0000  29.0000
4454      تفو   392.000000   392.000000  Default  28.0000  28.0000
6255       شي   371.000000   371.000000  Default  27.0000  27.0000
3002       ان   433.000000   433.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
5514     ديال    23.637354   285.001476   Topic5  -6.1311  -0.3915
3122       او    24.030936   426.970213   Topic5  -6.1146  -0.7792
1460   الدوله    18.213247    88.873238   Topic5  -6.3918   0.5131
6794    عليكم    18.645411   176.265290   Topic5  -6.3684  -0.1483
11437    يعني    17.841544   113.400105   Topic5  -6.4124   0.2487

[358 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
13         5  0.873139      20
34         3  0.970570  60درهم
80         3  0.917652   ابليس
81         2  0.176161     ابن
81         3  0.327157     ابن
...      ...       ...     ...
11653      1  0.170298     يوم
11653      2  0.099340     يوم
11653      3  0.170298     يوم
11653      4  0.170298     يوم
11653      5  0.383170     يوم

[574 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])